In [18]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import random_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [19]:
#Class: RNN
class RNN(nn.Module):
    #sizes are the number of nodes for respective layers, as ints
    #input_size should be the numbers of variables in input, output_size the number of variables predicted
    #hidden_size should be chosen after experimentation, since we only have one layer it should be more than the number of variables
    def __init__(self, input_size, hidden_size, output_size, **kwds):
        super().__init__(**kwds)
        #Number of nodes of the hidden layer (used for init)
        self.hidden_size = hidden_size
        #Weights
        self.i2h = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.h2o = nn.Linear(hidden_size, output_size)
    
    #Input must be torch.Tensor and normalized
    def forward(self, input):
        #h0 = self.initHidden()
        h_list, hn = self.i2h(input)
        output_list = self.h2o(h_list)
        return output_list

In [20]:
#Class : LSTM
class LSTM(nn.Module):
    #sizes are the number of nodes for respective layers, as ints
    #input_size should be the numbers of variables in input, output_size the number of variables predicted
    #hidden_size should be chosen after experimentation, since we only have one layer it should be more than the number of variables
    def __init__(self, input_size, hidden_size, output_size, **kwds):
        super().__init__(**kwds)
        #Number of nodes of the hidden layer (used for init)
        self.hidden_size = hidden_size
        #Weights
        self.i2h = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.h2o = nn.Linear(hidden_size, output_size)

    
    #Input must be torch.Tensor and normalized
    def forward(self, input):
        h_list, (hn,cn) = self.i2h(input)
        output_list = self.h2o(h_list)
        return output_list

In [21]:
#Parameters to get several scenari
CSV_FILE = "../../data/synthetic_bph_1/Simulations/simulation1.csv"
TRAINING_NUMBER = 400
USE_NOISY_DATA = False
USE_MIXED_EFFECT = False
timestep_skip = 1
MODEL = RNN        

In [22]:
#data loading
data = pd.read_csv(CSV_FILE, sep=";", decimal=",")

In [ ]:
x_labels = [
    c for c in data.columns if c.startswith("x") and (("_" in c) is USE_NOISY_DATA)
]
if 'x8' not in x_labels:
    x_labels.append('x8')
#assert len(x_labels) == 8

y_labels = [
    c
    for c in data.columns
    if c.startswith("y")
    and (("_obs" in c))
    and (("_mixed" in c) is USE_MIXED_EFFECT)
]
assert len(y_labels) == 1

print(x_labels)
print(y_labels)

In [24]:
# preprocessing
random.seed(0)
data = data[data['temps']%timestep_skip == 0]
data['y+1'] = data.groupby("individus")[y_labels].shift(-1)
data_norm = data.copy()
data_norm = data_norm.dropna()
N_train = random.sample(range(1,500), TRAINING_NUMBER)
N_train.sort()
data_train = data_norm.loc[data_norm['individus'].isin(N_train)]
data_test = data_norm[~data_norm['individus'].isin(N_train)]

scaler_x = MinMaxScaler()
data_train.loc[:,x_labels + y_labels] = scaler_x.fit_transform(data_train[x_labels + y_labels])
data_test.loc[:,x_labels + y_labels] = scaler_x.transform(data_test[x_labels + y_labels ])

scaler_y = MinMaxScaler()
data_train.loc[:,['y+1']] = scaler_y.fit_transform(data_train[['y+1']])
data_test.loc[:,['y+1']] = scaler_y.transform(data_test[['y+1']])

groupby = data_train.groupby('individus')[x_labels + y_labels].apply(np.array)
input_train = [torch.Tensor(x) for x in groupby]
input_train = torch.stack(input_train)
groupby = data_train.groupby('individus')[['y+1']].apply(np.array)
target_train = [torch.Tensor(x) for x in groupby]
target_train = torch.stack(target_train)

groupby = data_test.groupby('individus')[x_labels + y_labels].apply(np.array)
input_test = [torch.Tensor(x) for x in groupby]
input_test = torch.stack(input_test)
groupby = data_test.groupby('individus')[['y+1']].apply(np.array)
target_test = [torch.Tensor(x) for x in groupby]
target_test = torch.stack(target_test)
#seq_lens = [len(inp) for inp in groupby]
#input_padded = pad_sequence([torch.tensor(x) for x in groupby])
#input_pack_padded = pack_padded_sequence(input_padded, lengths=seq_lens)



In [25]:
#hyperparamètres
hidden_size = [25]
learning_rate = [ 0.001]
criterion = torch.nn.MSELoss()
epoch = 1000
eps = 0.0001

In [ ]:
#Training with convergence criteria and GridSearch
min_loss = float('inf')
for h in hidden_size:
    for lr in learning_rate:
        model = MODEL(input_size = 9, hidden_size=h, output_size=1)
        optimizer = torch.optim.SGD(model.parameters(), lr = lr)
        loss_val = [float('inf')]*100
        loss_train = []
        cur_loss_val = 0
        nb_epochs = 0
        while abs(np.max(loss_val[-100:] - np.min(loss_val[-100:] + [cur_loss_val]))) >= eps:

            y_test = model(input = input_test)
            cur_loss_val = criterion(y_test, target_test).item()
            loss_val.append(cur_loss_val)

            optimizer.zero_grad()
            y_pred = model(input=input_train)
            loss = criterion(y_pred, target_train)
            loss.backward()
            optimizer.step()

            cur_loss_train = loss.item()
            loss_train.append(cur_loss_train)
            
            nb_epochs += 1
        if cur_loss_val < min_loss:
            min_loss = cur_loss_val
            res = (h, lr)

loss_val = loss_val[101:]
print(h, lr)
print(nb_epochs)

In [ ]:
#plot loss
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.plot(loss_track)
plt.plot(loss_train, label ="loss")
plt.plot(loss_val, label = "validation")
plt.legend()
plt.show()

In [ ]:
print(criterion(y_pred,target_train).item())

In [ ]:
print(N_train)

In [30]:
#unscale data
multi_index = pd.MultiIndex.from_product([N_train, range(y_pred.shape[1])], names=['individus', 'temps'])
df = pd.DataFrame(index = multi_index, data = y_pred.detach().numpy().flatten(), columns=['y_pred'])
df = df.reset_index()
df = df.set_index(data_train.index)
df.loc[:,'y_pred'] = scaler_y.inverse_transform(df[['y_pred']])
data_train.loc[:,['y+1']] = scaler_y.inverse_transform(data_train[['y+1']])

In [ ]:
print(df[(df['temps']!=0) & (df['individus'] == 1)]['y_pred'])

In [ ]:
#Results on training
MSE_list = []
for k in N_train:
    pred_k = df[(df['temps']!=0) & (df['individus'] == k)]['y_pred']
    target_k = data_train[(data_train['temps']!=0) & (data_train['individus'] == k)]['y+1']
    MSE_list.append(mean_squared_error(pred_k, target_k))
print("mean MSE on train data: ", np.mean(MSE_list))

In [ ]:
#Plot 4 random curves from training data and worst individual
n = 5
ex = randint(1,400, n-1)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(data_train.loc[(data_train['temps']!=0) & (data_train['individus']==k)]['y+1'], label= 'target')
    axs[k].plot(df.loc[(df['temps']!=0) & (df['individus']==k)]['y_pred'], label= 'prediction')
    axs[k].legend()

In [ ]:
#MSE on test data
y_pred = model(input_test)
#Results
MSE_list = []
for k in range(len(y_pred)):
    MSE_list.append(mean_squared_error(y_pred[k, 1:].detach().numpy(),target_test[k, 1:].detach().numpy()))
print("mean MSE on test data: ", np.mean(MSE_list))
print("Worst case :", np.max(MSE_list), "for individual ", np.argmax(MSE_list))

In [ ]:
#Plot n curves from test data
n = 10
ex = randint(1,100, n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
res = []
print(y_pred.shape)
for k in range(n):
    res.append((ex[k], mean_squared_error(target_test[ex[k], 1:].detach().numpy(), y_pred[ex[k], 1:].detach().numpy())))
    axs[k].plot(target_test[ex[k], 1:].detach().numpy(), label= 'target')
    axs[k].plot(y_pred[ex[k], 1:].detach().numpy(), label= 'prediction')
    axs[k].plot(torch.roll(target_test, 1, 1)[ex[k], 1:], label = 'reporté de %timestepskip jours')
    axs[k].legend()

print(res) #res contient les MSE

In [ ]:
#plot n worst curves
indices = np.argpartition(MSE_list, -5)[-5:]
n = 5
ex = randint(1,100, n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(len(indices)):
    axs[k].plot(target_test[indices[k], 1:].detach().numpy(), label= 'target')
    axs[k].plot(y_pred[indices[k], 1:].detach().numpy(), label= 'prediction')
    axs[k].plot(torch.roll(target_test, 1, 1)[indices[k], 1:], label = 'reporté de %timestepskip jours')
    axs[k].legend()